## netmamba embeddings

In [ ]:
%cd ../models/NetMamba/src

/pscratch/sd/k/kell/demystifying/NetMamba/src


/global/homes/k/kell/scratch/demystifying/netmambaenv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from torchvision import datasets, transforms
import torch
import models_net_mamba
from util.pos_embed import interpolate_pos_embed
from timm.models.layers import trunc_normal_
import os
from tqdm import tqdm
import copy
from collections import defaultdict
import threading
import pickle

/global/homes/k/kell/scratch/demystifying/netmambaenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
os.environ['PATH'] = '/sbin:' + os.environ.get('PATH', '')

In [ ]:
def build_dataset(data_path):
    mean = [0.5]
    std = [0.5]

    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
    dataset = datasets.ImageFolder(data_path, transform=transform)
    return dataset

def get_embeddings(datafolder, batch_size=64, limit = 10**30, gpus=4):
    dataset = build_dataset(datafolder)
    sampler = torch.utils.data.SequentialSampler(dataset)
    dataloader = torch.utils.data.DataLoader(
        dataset, sampler=sampler,
        batch_size=batch_size,
        num_workers=8
    )
    model = models_net_mamba.__dict__['net_mamba_classifier'](
        num_classes=2,
        drop_path_rate=0,
    )

    checkpoint = torch.load('../models/NetMamba/pre-train.pth', map_location='cpu')
    checkpoint_model = checkpoint['model']
    state_dict = model.state_dict()
    for k in ['head.weight', 'head.bias']:
        if k in checkpoint_model and checkpoint_model[k].shape != state_dict[k].shape:
            print(f"Removing key {k} from pretrained checkpoint")
            del checkpoint_model[k]
    
    # interpolate position embedding
    interpolate_pos_embed(model, checkpoint_model)
    
    # load pre-trained model
    msg = model.load_state_dict(checkpoint_model, strict=False)
    print(msg)
    
    # manually initialize fc layer
    trunc_normal_(model.head.weight, std=2e-5)

    models = {}
    for i in range(gpus):
        models[i] = copy.deepcopy(model)
        models[i].to(f"cuda:{i}")
    
    def encode_and_append(batch, model, result_list, i):
        imgs, _ = batch
        with torch.no_grad():
            result_list.append(model.forward_encoder(imgs.to(f"cuda:{i}"), mask_ratio=0.0, if_mask=False)[:, -1, :].cpu())
        del imgs
        del batch

    counter = 0
    result_embeddings = []
    result_filenames = []
    with torch.no_grad():
        iterator = iter(dataloader)
        try:
            for y in tqdm(range(0, min(len(dataloader) // gpus, limit))):
                emb = defaultdict(list)
                batches = [next(iterator) for i in range(gpus)]
                threads = []
                for i in range(gpus):
                    t = threading.Thread(target=encode_and_append, args=(batches[i], models[i], emb[i], i))
                    t.start()
                    threads.append(t)
                for t in threads:
                    t.join()
                del batches
                result_embeddings.append(torch.cat([torch.cat(emb[i]) for i in emb]))
                result_filenames.extend([x[0] for x in dataloader.dataset.samples[y*gpus*batch_size:(y+1)*gpus*batch_size]])
                
        except StopIteration:
            print("finished")
        except Exception as e:
            print(e)

    return torch.cat(result_embeddings), result_filenames
    

In [7]:
labels = ['cross', 'cicids', 'cicapt', 'caida', 'mawi']

In [8]:
labels = ['synth']

In [ ]:
for label in labels:
    print(label)
    emb = get_embeddings(f"../data/{label}/array_sampled", batch_size=512)
    with open(f"../data/netmamba_{label}_emb.pkl", "bw") as f:
        pickle.dump(emb, f)

perf
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=['mask_token', 'decoder_pos_embed', 'decoder_embed.weight', 'decoder_embed.bias', 'decoder_blocks.0.mixer.A_log', 'decoder_blocks.0.mixer.D', 'decoder_blocks.0.mixer.in_proj.weight', 'decoder_blocks.0.mixer.conv1d.weight', 'decoder_blocks.0.mixer.conv1d.bias', 'decoder_blocks.0.mixer.x_proj.weight', 'decoder_blocks.0.mixer.dt_proj.weight', 'decoder_blocks.0.mixer.dt_proj.bias', 'decoder_blocks.0.mixer.out_proj.weight', 'decoder_blocks.0.norm.weight', 'decoder_blocks.1.mixer.A_log', 'decoder_blocks.1.mixer.D', 'decoder_blocks.1.mixer.in_proj.weight', 'decoder_blocks.1.mixer.conv1d.weight', 'decoder_blocks.1.mixer.conv1d.bias', 'decoder_blocks.1.mixer.x_proj.weight', 'decoder_blocks.1.mixer.dt_proj.weight', 'decoder_blocks.1.mixer.dt_proj.bias', 'decoder_blocks.1.mixer.out_proj.weight', 'decoder_blocks.1.norm.weight', 'decoder_norm_f.weight', 'decoder_pred.weight', 'decoder_pred.bias'])


0it [00:00, ?it/s]


RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [ ]:
label = "synth"
emb = get_embeddings(f"../data/{label}/array_sampled", batch_size=1, gpus=1)
with open(f"../data/netmamba_{label}_emb.pkl", "bw") as f:
    pickle.dump(emb, f)

_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=['mask_token', 'decoder_pos_embed', 'decoder_embed.weight', 'decoder_embed.bias', 'decoder_blocks.0.mixer.A_log', 'decoder_blocks.0.mixer.D', 'decoder_blocks.0.mixer.in_proj.weight', 'decoder_blocks.0.mixer.conv1d.weight', 'decoder_blocks.0.mixer.conv1d.bias', 'decoder_blocks.0.mixer.x_proj.weight', 'decoder_blocks.0.mixer.dt_proj.weight', 'decoder_blocks.0.mixer.dt_proj.bias', 'decoder_blocks.0.mixer.out_proj.weight', 'decoder_blocks.0.norm.weight', 'decoder_blocks.1.mixer.A_log', 'decoder_blocks.1.mixer.D', 'decoder_blocks.1.mixer.in_proj.weight', 'decoder_blocks.1.mixer.conv1d.weight', 'decoder_blocks.1.mixer.conv1d.bias', 'decoder_blocks.1.mixer.x_proj.weight', 'decoder_blocks.1.mixer.dt_proj.weight', 'decoder_blocks.1.mixer.dt_proj.bias', 'decoder_blocks.1.mixer.out_proj.weight', 'decoder_blocks.1.norm.weight', 'decoder_norm_f.weight', 'decoder_pred.weight', 'decoder_pred.bias'])


100%|██████████| 472/472 [00:05<00:00, 89.98it/s] 
